In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import googlemaps
from datetime import datetime

In [44]:
# Funcion para obtener direcciones de Google API

gmaps = googlemaps.Client(key='*****') #---- Ingresar API KEY

def obtener_dir(lat, lng, gmaps):
    try:
        reverse_geocode_result = gmaps.reverse_geocode((lat, lng))
        return reverse_geocode_result[0]['formatted_address']
    except Exception as e:
        print(f"Error al obtener la dirección: {e}")
        return None


## EXTRACTION

In [3]:
# Lectura Dataframes

df_hom_hec = pd.read_excel(r'..\Raw Data\homicidios.xlsx', sheet_name= 'HECHOS') 
df_hom_vic = pd.read_excel(r'..\Raw Data\homicidios.xlsx', sheet_name= 'VICTIMAS') 

df_les_hec = pd.read_excel(r'..\Raw Data\lesiones.xlsx', sheet_name= 'HECHOS')
df_les_vic = pd.read_excel(r'..\Raw Data\lesiones.xlsx', sheet_name= 'VICTIMAS') 


## TRANSFORM

### Dataset Hechos Homicidios

In [4]:
df_hom_hec.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [5]:
list(df_hom_hec.columns)

['ID',
 'N_VICTIMAS',
 'FECHA',
 'AAAA',
 'MM',
 'DD',
 'HORA',
 'HH',
 'LUGAR_DEL_HECHO',
 'TIPO_DE_CALLE',
 'Calle',
 'Altura',
 'Cruce',
 'Dirección Normalizada',
 'COMUNA',
 'XY (CABA)',
 'pos x',
 'pos y',
 'PARTICIPANTES',
 'VICTIMA',
 'ACUSADO']

In [6]:
# Drop de columnas innecesarias

df_hom_hec1 = df_hom_hec.copy()
 
df_hom_hec1 = df_hom_hec1.drop(['AAAA',
                              'MM',
                              'DD',
                              'HH',
                              'Calle',
                              'Altura',
                              'Cruce',
                              'Dirección Normalizada',
                              'XY (CABA)',
                              'PARTICIPANTES'],
                              axis = 1
)

list(df_hom_hec1.columns)


['ID',
 'N_VICTIMAS',
 'FECHA',
 'HORA',
 'LUGAR_DEL_HECHO',
 'TIPO_DE_CALLE',
 'COMUNA',
 'pos x',
 'pos y',
 'VICTIMA',
 'ACUSADO']

In [7]:
df_hom_hec1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ID               696 non-null    object        
 1   N_VICTIMAS       696 non-null    int64         
 2   FECHA            696 non-null    datetime64[ns]
 3   HORA             696 non-null    object        
 4   LUGAR_DEL_HECHO  696 non-null    object        
 5   TIPO_DE_CALLE    696 non-null    object        
 6   COMUNA           696 non-null    int64         
 7   pos x            696 non-null    object        
 8   pos y            696 non-null    object        
 9   VICTIMA          696 non-null    object        
 10  ACUSADO          696 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(8)
memory usage: 59.9+ KB


In [8]:
# Revision de datos duplicados

df_hom_hec1.duplicated().sum()

0

In [9]:
# Revision de datos nulos (Luego se revisará ya que el dataset tiene valores nulos con string 'SD')

df_hom_hec1.isna().sum().sum()

0

In [10]:
# Transformar todos los strings a lowercase
columnas = df_hom_hec1.columns.tolist()
columnas_excluidas = ['N_VICTIMAS','FECHA','HORA','COMUNA','pos x','pos y']
columnas_incluidas = [columna for columna in columnas if columna not in columnas_excluidas]

for columna in columnas_incluidas:
    df_hom_hec1[columna] = df_hom_hec1[columna].str.lower()


In [11]:
# Transformar COMUNA a string

df_hom_hec1['COMUNA'] = df_hom_hec1['COMUNA'].astype('str')


In [12]:
# Drop de valores con '.' de la columna 'pos x'

'''
Tanto la columna pos x como la columna pos y se utilizaran para traer las direcciones desde la api de google. 
En este caso, se eliminaran aquellos registros que no muestren valores en este punto (representan aproxiadamente el 2% de los datos)
'''


print('Antes ', (df_hom_hec1['pos x'] == '.').value_counts())

df_hom_hec1 = df_hom_hec1[df_hom_hec1['pos x'] != '.']

print('Despues ', (df_hom_hec1['pos x'] == '.').value_counts())

Antes  pos x
False    684
True      12
Name: count, dtype: int64
Despues  pos x
False    684
Name: count, dtype: int64


In [13]:
# Ajustando nombre de columnas

nom_col_hom_hec1 = [
    'ID_HECHO', 
    'N_VICTIMAS', 
    'FECHA', 
    'HORA', 
    'LUGAR_DEL_HECHO', 
    'TIPO_DE_CALLE', 
    'COMUNA', 
    'LONGITUD', 
    'LATITUD', 
    'VICTIMA', 
    'ACUSADO'
]

df_hom_hec1.columns = nom_col_hom_hec1

In [45]:
# Obtencion de direcciones desde API de Google, con base en Latitud y Longitud

df_hom_hec1['DIRECCION'] = df_hom_hec1.apply(lambda row: obtener_dir(row['LATITUD'], row['LONGITUD'], gmaps), axis=1)


In [46]:
df_hom_hec1.head()

,ID_HECHO,N_VICTIMAS,FECHA,HORA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,LONGITUD,LATITUD,VICTIMA,ACUSADO,DIRECCION
0,2016-0001,1,2016-01-01,04:00:00,av piedra buena y av fernandez de la cruz,avenida,8,-58.47533969,-34.68757022,moto,auto,"Av. Piedra Buena, Av. Gral. Francisco Fernánde..."
1,2016-0002,1,2016-01-02,01:15:00,av gral paz y av de los corrales,gral paz,9,-58.50877521,-34.66977709,auto,pasajeros,"De la Torre Lisandro 1684, B1751 La Tablada, P..."
2,2016-0003,1,2016-01-03,07:00:00,av entre rios 2034,avenida,1,-58.39040293,-34.63189362,moto,auto,"Av. Entre Ríos 2036, C1133AAV Buenos Aires, Ar..."
3,2016-0004,1,2016-01-10,00:00:00,av larrazabal y gral villegas conrado,avenida,8,-58.46503904,-34.68092974,moto,sd,"Larrazabal, Buenos Aires, Argentina"
4,2016-0005,1,2016-01-21,05:20:00,av san juan y presidente luis saenz peña,avenida,1,-58.38718297,-34.62246630,moto,pasajeros,"1212 Saenz Pe?a Luis Pres., C1148 Buenos Aires..."


### Dataset Victimas Homicidios

In [15]:
df_hom_vic.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [16]:
list(df_hom_vic.columns)

['ID_hecho',
 'FECHA',
 'AAAA',
 'MM',
 'DD',
 'ROL',
 'VICTIMA',
 'SEXO',
 'EDAD',
 'FECHA_FALLECIMIENTO']

In [17]:
# Drop de columnas innecesarias

df_hom_vic1 = df_hom_vic.copy()
 
df_hom_vic1 = df_hom_vic1.drop(['FECHA',
                              'AAAA',
                              'MM',
                              'DD',
                              'VICTIMA'],
                              axis = 1
)

list(df_hom_vic1.columns)


['ID_hecho', 'ROL', 'SEXO', 'EDAD', 'FECHA_FALLECIMIENTO']

In [18]:
# Revision de datos duplicados

df_hom_vic1.duplicated().sum()

0

In [19]:
# Revision de datos nulos (Luego se revisará ya que el dataset tiene valores nulos con string 'SD')

df_hom_vic1.isna().sum().sum()

0

In [20]:
# Transformar todos los strings a lowercase

df_hom_vic1['ROL'] = df_hom_vic1['ROL'].str.lower()
df_hom_vic1['SEXO'] = df_hom_vic1['SEXO'].str.lower()


In [21]:
# Reemplazar 'SD' con NaN en la columna 'FECHA_FALLECIMIENTO'
df_hom_vic1['FECHA_FALLECIMIENTO'] = df_hom_vic1['FECHA_FALLECIMIENTO'].replace('SD', np.nan)

# Extraer solo la fecha de la columna 'FECHA_FALLECIMIENTO' y crear una nueva columna 'FECHA'
df_hom_vic1['FECHA_FALLECIMIENTO'] = pd.to_datetime(df_hom_vic1['FECHA_FALLECIMIENTO']).dt.date



In [22]:
# Ajustando nombre de columnas

nom_col_hom_vic1 = [
    'ID_HECHO',
    'ROL',
    'SEXO',
    'EDAD',
    'FECHA_FALLECIMIENTO'
]

df_hom_vic1.columns = nom_col_hom_vic1

In [23]:
df_hom_vic1.head()


,ID_HECHO,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,conductor,masculino,19,2016-01-01
1,2016-0002,conductor,masculino,70,2016-01-02
2,2016-0003,conductor,masculino,30,2016-01-03
3,2016-0004,conductor,masculino,18,NaT
4,2016-0005,conductor,masculino,29,2016-02-01


### Dataset Hechos Lesiones

In [24]:
df_les_hec.head()

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,GRAVEdad
0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9,SD,14,...,-34.559658,CICLISTA,SD,CICLISTA-SD,SD,SD,SD,SD,x,SD
1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1,SD,8,...,-34.669125,AUTO,SD,AUTO-SD,SD,x,SD,SD,SD,SD
2,LC-2019-0000063,1,2019,1,1,2019-01-01 00:00:00,02:00:00,2,SD,8,...,-34.677556,SD,SD,SD-SD,SD,SD,SD,SD,SD,SD
3,LC-2019-0000079,1,2019,1,1,2019-01-01 00:00:00,02:30:00,2,SD,7,...,-34.647349,PEATON,SD,PEATON-SD,x,SD,SD,SD,SD,SD
4,LC-2019-0000082,4,2019,1,1,2019-01-01 00:00:00,04:30:00,4,SD,3,...,-34.604579,AUTO,SD,AUTO-SD,SD,SD,x,SD,SD,SD


In [25]:
list(df_les_hec.columns)

['id',
 'n_victimas',
 'aaaa',
 'mm',
 'dd',
 'fecha',
 'hora',
 'franja_hora',
 'direccion_normalizada',
 'comuna',
 'tipo_calle',
 'otra_direccion',
 'calle',
 'altura',
 'cruce',
 'geocodificacion_CABA',
 'longitud',
 'latutid',
 'victima',
 'acusado',
 'participantes',
 'moto',
 'auto',
 'transporte_publico',
 'camion',
 'ciclista',
 'GRAVEdad']

In [26]:
#Drop de columnas innecesarias 
df_les_hec1 = df_les_hec.drop(['aaaa',
                               'mm',
                               'dd',
                               'franja_hora',
                               'direccion_normalizada',
                               'calle',
                               'altura',
                               'cruce',
                               'geocodificacion_CABA',
                               'participantes',
                               'moto',
                               'auto',
                               'transporte_publico',
                               'camion',
                               'ciclista',],
                               axis = 1
)


df_les_hec1.columns = df_les_hec1.columns.str.upper()

In [27]:
# Drop de valores con 'SD' de la columna 'longitud'

'''
Tanto la columna longitud como la columna latitud se utilizaran para traer las direcciones desde la api de google. 
En este caso, se eliminaran aquellos registros que no muestren valores en este punto (represetan aproximadamete el 5% de los datos)
'''


print('Antes ', (df_les_hec1['LONGITUD'] == 'SD').value_counts())

df_les_hec1 = df_les_hec1[df_les_hec1['LONGITUD'] != 'SD']

print('Despues ', (df_les_hec1['LONGITUD'] == 'SD').value_counts())

Antes  LONGITUD
False    22576
True      1209
Name: count, dtype: int64
Despues  LONGITUD
False    22576
Name: count, dtype: int64


In [28]:
# Corregir formato de fechas

df_les_hec1['FECHA'] = pd.to_datetime(df_les_hec1['FECHA']).dt.strftime('%Y-%m-%d')


In [29]:
list(df_les_hec1.columns)

['ID',
 'N_VICTIMAS',
 'FECHA',
 'HORA',
 'COMUNA',
 'TIPO_CALLE',
 'OTRA_DIRECCION',
 'LONGITUD',
 'LATUTID',
 'VICTIMA',
 'ACUSADO',
 'GRAVEDAD']

In [30]:
# Ajustando nombre de columnas

nom_col_les_hec1 = [
    'ID_HECHO',
    'N_VICTIMAS',
    'FECHA',
    'HORA',
    'COMUNA',
    'TIPO_CALLE',
    'DIRECCION',
    'LONGITUD',
    'LATITUD',
    'VICTIMA',
    'ACUSADO',
    'GRAVEDAD'
]

df_les_hec1.columns = nom_col_les_hec1

In [31]:
df_les_hec1.head()

,ID_HECHO,N_VICTIMAS,FECHA,HORA,COMUNA,TIPO_CALLE,DIRECCION,LONGITUD,LATITUD,VICTIMA,ACUSADO,GRAVEDAD
0,LC-2019-0000179,1,2019-01-01,09:00:00,14,SD,"Obligado Rafael, Av.Costanera 3300, Barrio: Pa...",-58.408911,-34.559658,CICLISTA,SD,SD
1,LC-2019-0000053,1,2019-01-01,01:55:00,8,SD,"Autopista Presidente Héctor J. Cámpora y Roca,...",-58.44351,-34.669125,AUTO,SD,SD
2,LC-2019-0000063,1,2019-01-01,02:00:00,8,SD,"Barros Pazos, Jose y Larraya 0, Barrio: Villa ...",-58.468335,-34.677556,SD,SD,SD
3,LC-2019-0000079,1,2019-01-01,02:30:00,7,SD,"Torres Y Tenorio, Pres. y Riestra Av. 0, Barri...",-58.437425,-34.647349,PEATON,SD,SD
4,LC-2019-0000082,4,2019-01-01,04:30:00,3,SD,"Corrientes Av. y Uriburu Jose E., Pres. 0, Bar...",-58.398225,-34.604579,AUTO,SD,SD


### Dataset Victimas Lesiones

In [32]:
df_les_vic.head()

,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,2019-01-01,sd,Varon,57,SD
1,LC-2019-0000063,2019,1,1,2019-01-01,sd,SD,SD,SD
2,LC-2019-0000079,2019,1,1,2019-01-01,sd,Varon,SD,SD
3,LC-2019-0000082,2019,1,1,2019-01-01,sd,Varon,45,SD
4,LC-2019-0000082,2019,1,1,2019-01-01,sd,Mujer,45,SD


In [33]:
list(df_les_vic.columns)

['ID hecho',
 'AAA',
 'MM',
 'DD',
 'FECHA ',
 'VEHICULO_VICTIMA',
 'SEXO',
 'EDAD_VICTIMA',
 'GRAVEDAD']

In [34]:
# Drop de columnas innecesarias

df_les_vic1 = df_les_vic.copy()
 
df_les_vic1 = df_les_vic1.drop(['FECHA ',
                              'AAA',
                              'MM',
                              'DD',
                              'VEHICULO_VICTIMA'],
                              axis = 1
)

list(df_les_vic1.columns)

['ID hecho', 'SEXO', 'EDAD_VICTIMA', 'GRAVEDAD']

In [35]:
# Ajustando nombre de columnas

nom_col_les_vic1 = [
    'ID_HECHO',
    'SEXO',
    'EDAD',
    'GRAVEDAD'
]

df_les_vic1.columns = nom_col_les_vic1

In [36]:
# Transformar todos los strings a lowercase
columnas = df_les_vic1.columns.tolist()
columnas_excluidas = ['ID_HECHO','EDAD']
columnas_incluidas = [columna for columna in columnas if columna not in columnas_excluidas]

for columna in columnas_incluidas:
    df_les_vic1[columna] = df_les_vic1[columna].str.lower()

In [37]:
df_les_vic1.head()

,ID_HECHO,SEXO,EDAD,GRAVEDAD
0,LC-2019-0000053,varon,57,sd
1,LC-2019-0000063,sd,SD,sd
2,LC-2019-0000079,varon,SD,sd
3,LC-2019-0000082,varon,45,sd
4,LC-2019-0000082,mujer,45,sd


### Dataset Poblacion

In [38]:
# Ruta al archivo Excel
file_path = r"..\Raw Data\poblacion.xlsx"

# Crear un DataFrame vacío para recopilar los datos de todas las hojas
df_poblacion = pd.DataFrame()

# Identificar Columnas de Comunas
columnas_comunas = ['C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q']

# Procesar cada hoja por separado
for año in range(2016, 2022):
    # Leer la hoja correspondiente al año, las comunas están en las columnas 'C' a 'Q'
    df_año = pd.read_excel(file_path, sheet_name=str(año), usecols='C:Q', skiprows=2)

    # extraccion de primer fila (total de poblacion)
    total_poblacion = df_año.iloc[0].values


    # Crear un DataFrame con los datos de población para este año
    df_population = pd.DataFrame({
        'AÑO': año,
        'COMUNA': range(1,16),
        'POBLACION': total_poblacion
    })

    # Añadir los datos de cada año al DataFrame
    df_poblacion = pd.concat([df_poblacion, df_population], ignore_index=True)


In [39]:
df_poblacion

,AÑO,COMUNA,POBLACION
0,2016,1,252053.0
1,2016,2,149848.0
2,2016,3,192573.0
3,2016,4,238303.0
4,2016,5,186740.0
...,...,...,...
85,2021,11,190105.0
86,2021,12,215002.0
87,2021,13,236468.0
88,2021,14,227168.0


## LOAD

In [40]:
# Credenciales de MySQL
username = '****' #----> ingresar datos
password = '****' #----> ingresar datos
host = '****' #----> ingresar datos
port = '****' #----> ingresar datos
database = 'acc_vial'

# Crear motor de conexion
engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}')


In [47]:
# Ingesta de datos en MySQL

df_hom_hec1.to_sql('hechos_homicidios', con=engine, if_exists='append', index=False)
df_hom_vic1.to_sql('victimas_homicidios', con=engine, if_exists='append', index=False)
df_les_hec1.to_sql('hechos_lesiones', con=engine, if_exists='append', index=False)
df_les_vic1.to_sql('victimas_lesiones', con=engine, if_exists='append', index=False)
df_poblacion.to_sql('poblacion', con=engine, if_exists='append', index=False)

90